In [10]:
%load_ext autoreload
%autoreload 2

import os
import sys

sys.path.append(os.path.join(os.getcwd()[:os.getcwd().find('notebooks')]))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
import string
import numpy as np
import pandas as pd
from collections import defaultdict

from config import cfg

base_path = os.path.join(os.getcwd()[:os.getcwd().find('notebooks')])

### Pairs

In [19]:
pairs = pd.read_csv(os.path.join(cfg["docs_path"], "bld_pairs.csv"), index_col=0)

In [47]:
pairs_long = (
    pairs
    .melt(ignore_index=False)
    .reset_index()
    .dropna()
    .assign(letter_pair=lambda df: df["index"] + df["variable"])
    .rename(columns={"value": "image"})
    .pipe(lambda df: df[["letter_pair", "image"]])
    .pipe(lambda df: df[df.image != "."])
    # .to_csv(os.path.join(cfg["docs_path"], "bld_pairs_long.csv"))
)

In [51]:
pairs_long.pipe(lambda df: df[df.image.str.contains("book")])

,letter_pair,image
30,EB,Ebook
31,FB,Facebook
